In [1]:
import psycopg2

from sqlalchemy import create_engine
from sqlalchemy import text

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
database_host='localhost'

In [3]:
def get_connection(host = "localhost",schema="" ):
    
    conn = psycopg2.connect("dbname='proj001_lfb' user='postgres' "
                            "host=" + host )
    if schema:
        cur = conn.cursor()
        query = "SET search_path TO {}".format(schema)
        cur.execute(query)

    return conn

# Create a feature dataset

## Incidents

In [4]:
conn = get_connection(host=database_host)
qry = ("select incidents.*, types.incidentgroup, types.stopcodedescription "
          "from proj001_lfb_1_0_0.l3_incidents incidents "
           "left join "
           "proj001_lfb_1_0_0.l3_incident_types types "
           "on "
           "incidents.id_incident_type = types.id"
          ";")
#data = pd.read_sql(qry, conn).drop(['store_no','store_type'], axis=1)  
df_incidents = pd.read_sql(qry, conn)  
conn.close()

# drop some columns that won't be features
df_incidents.drop(['num_stations_with_pumps_attending', 'num_pumps_attending'], axis =1 ,inplace=True)
df_incidents.head()

,id,incidentnumber_cln,dateofcall_cln,id_incident_type,id_property,id_postcode,incident_station_ground,incidentgroup,stopcodedescription
0,1,79398141,2014-06-22 10:21:15,2,24,108386,OVERTHEBORDER,False Alarm,False alarm - Good intent
1,2,168174131,2013-12-07 04:29:07,6,24,108387,OVERTHEBORDER,Fire,Primary Fire
2,3,168954111,2011-10-16 12:20:54,1,32,108388,BROMLEY,False Alarm,AFA
3,4,168866-17122017,2017-12-17 13:06:19,1,29,108388,BROMLEY,False Alarm,AFA
4,5,167600111,2011-10-14 11:58:03,1,24,108388,BROMLEY,False Alarm,AFA


## Postcodes

In [5]:
conn = get_connection(host=database_host)
qry = ('select *'
          'from proj001_lfb_1_0_0.l3_postcodes '
          ';')
#data = pd.read_sql(qry, conn).drop(['store_no','store_type'], axis=1)  
df_postcodes = pd.read_sql(qry, conn)  
conn.close()
df_postcodes.head()

,id,postcode_full,postcode_district,incgeo_boroughname,incgeo_wardname,incgeo_wardnamenew,frs
0,1,AL1 3EA,AL1,NOT GEO-CODED,NOT GEO-CODED,NOT GEO-CODED,OVERTHEBORDER
1,2,AL2 1QY,AL2,NOT GEO-CODED,NOT GEO-CODED,NOT GEO-CODED,OVERTHEBORDER
2,3,AL8 7QA,AL8,NOT GEO-CODED,NOT GEO-CODED,NOT GEO-CODED,OVERTHEBORDER
3,4,BR1 1AB,BR1,BROMLEY,BROMLEY TOWN,BROMLEY TOWN,LONDON
4,5,BR1 1AE,BR1,BROMLEY,BROMLEY TOWN,BROMLEY TOWN,LONDON


Here we discover that fields may need to be stripped of trailing spaces in the pipeline code and that a proportion of the postcodes are unlabelled.

In [6]:
df_empty_pcodes = df_postcodes[df_postcodes['incgeo_boroughname']
             .map(lambda x: x.strip()) == 'NOT GEO-CODED'][['postcode_full','incgeo_boroughname']]['postcode_full'].unique()
print("Unlabeled postcodes: " + str(len(pd.DataFrame(df_empty_pcodes)) ))
print("Total postcodes: " + str(len(df_postcodes) ))

Unlabeled postcodes: 1669
Total postcodes: 110157


In [7]:
df_postcodes.drop(['postcode_district','incgeo_wardname','incgeo_wardnamenew'], inplace=True, axis =1 )
df_postcodes.head(2)

,id,postcode_full,incgeo_boroughname,frs
0,1,AL1 3EA,NOT GEO-CODED,OVERTHEBORDER
1,2,AL2 1QY,NOT GEO-CODED,OVERTHEBORDER


## Property details

In [8]:
conn = get_connection(host=database_host)
qry = ('select *'
          'from proj001_lfb_1_0_0.l3_property '
          ';')
#data = pd.read_sql(qry, conn).drop(['store_no','store_type'], axis=1)  
df_property = pd.read_sql(qry, conn)  
conn.close()
df_property.head(3)

,id,propertycategory,propertytype
0,1,Aircraft,Freight plane
1,2,Aircraft,Helicopter
2,3,Aircraft,Light aircraft


# Features dataset

In [16]:
df_features = pd.merge(pd.merge(df_incidents,df_postcodes,left_on='id_postcode',right_on='id', how='left') , 
                       df_property,left_on='id_property',right_on='id',how='left')
df_features.drop(['id','id_y'], inplace=True, axis =1)
df_features.head()

,id_x,incidentnumber_cln,dateofcall_cln,id_incident_type,id_property,id_postcode,incident_station_ground,incidentgroup,stopcodedescription,postcode_full,incgeo_boroughname,frs,propertycategory,propertytype
0,1,79398141,2014-06-22 10:21:15,2,24,108386,OVERTHEBORDER,False Alarm,False alarm - Good intent,None,NOT GEO-CODED,OVERTHEBORDER,Dwelling,House - single occupancy
1,2,168174131,2013-12-07 04:29:07,6,24,108387,OVERTHEBORDER,Fire,Primary Fire,None,NOT GEO-CODED,OVERTHEBORDER,Dwelling,House - single occupancy
2,3,168954111,2011-10-16 12:20:54,1,32,108388,BROMLEY,False Alarm,AFA,None,BROMLEY,LONDON,Dwelling,Self contained Sheltered Housing
3,4,168866-17122017,2017-12-17 13:06:19,1,29,108388,BROMLEY,False Alarm,AFA,None,BROMLEY,LONDON,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys
4,5,167600111,2011-10-14 11:58:03,1,24,108388,BROMLEY,False Alarm,AFA,None,BROMLEY,LONDON,Dwelling,House - single occupancy


In [21]:
df_features.columns.tolist
cols = ['id_x', 'dateofcall_cln','incidentnumber_cln', 'id_incident_type',
       'id_property', 'id_postcode', 'incident_station_ground',
     'postcode_full',
       'incgeo_boroughname', 'frs', 'propertycategory', 'propertytype', 'incidentgroup','stopcodedescription']
df_features=df_features[cols]
df_features.head()

,id_x,dateofcall_cln,incidentnumber_cln,id_incident_type,id_property,id_postcode,incident_station_ground,postcode_full,incgeo_boroughname,frs,propertycategory,propertytype,incidentgroup,stopcodedescription
0,1,2014-06-22 10:21:15,79398141,2,24,108386,OVERTHEBORDER,None,NOT GEO-CODED,OVERTHEBORDER,Dwelling,House - single occupancy,False Alarm,False alarm - Good intent
1,2,2013-12-07 04:29:07,168174131,6,24,108387,OVERTHEBORDER,None,NOT GEO-CODED,OVERTHEBORDER,Dwelling,House - single occupancy,Fire,Primary Fire
2,3,2011-10-16 12:20:54,168954111,1,32,108388,BROMLEY,None,BROMLEY,LONDON,Dwelling,Self contained Sheltered Housing,False Alarm,AFA
3,4,2017-12-17 13:06:19,168866-17122017,1,29,108388,BROMLEY,None,BROMLEY,LONDON,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,False Alarm,AFA
4,5,2011-10-14 11:58:03,167600111,1,24,108388,BROMLEY,None,BROMLEY,LONDON,Dwelling,House - single occupancy,False Alarm,AFA


# Create derived variables

In [27]:
df_features['incident_month']=df_features['dateofcall_cln'].dt.month
df_features['incident_year']=df_features['dateofcall_cln'].dt.year
df_features['incident_hour']=df_features['dateofcall_cln'].dt.hour
df_features['incident_day']=df_features['dateofcall_cln'].dt.dayofweek
df_features.head()

,id_x,dateofcall_cln,incidentnumber_cln,id_incident_type,id_property,id_postcode,incident_station_ground,postcode_full,incgeo_boroughname,frs,propertycategory,propertytype,incidentgroup,stopcodedescription,incident_month,incident_year,incident_hour,incident_day
0,1,2014-06-22 10:21:15,79398141,2,24,108386,OVERTHEBORDER,None,NOT GEO-CODED,OVERTHEBORDER,Dwelling,House - single occupancy,False Alarm,False alarm - Good intent,6,2014,10,6
1,2,2013-12-07 04:29:07,168174131,6,24,108387,OVERTHEBORDER,None,NOT GEO-CODED,OVERTHEBORDER,Dwelling,House - single occupancy,Fire,Primary Fire,12,2013,4,5
2,3,2011-10-16 12:20:54,168954111,1,32,108388,BROMLEY,None,BROMLEY,LONDON,Dwelling,Self contained Sheltered Housing,False Alarm,AFA,10,2011,12,6
3,4,2017-12-17 13:06:19,168866-17122017,1,29,108388,BROMLEY,None,BROMLEY,LONDON,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,False Alarm,AFA,12,2017,13,6
4,5,2011-10-14 11:58:03,167600111,1,24,108388,BROMLEY,None,BROMLEY,LONDON,Dwelling,House - single occupancy,False Alarm,AFA,10,2011,11,4


# Create the event label

In [37]:
df_features[['incidentgroup','stopcodedescription']].drop_duplicates().sort_values(by='incidentgroup')

,incidentgroup,stopcodedescription
0,False Alarm,False alarm - Good intent
2,False Alarm,AFA
65,False Alarm,False alarm - Malicious
1,Fire,Primary Fire
14,Fire,Chimney Fire
571,Fire,Secondary Fire
3127,Fire,Late Call
452853,NULL,NULL
11,Special Service,Special Service
42,Special Service,Flood call attended - Batch mobilised


In [41]:
df_features['is_malicious_false_alarm'] = np.where(df_features['stopcodedescription'] == 'False alarm - Malicious', 1,0)
df_features[['incidentgroup','stopcodedescription','is_malicious_false_alarm']].drop_duplicates().sort_values(by='incidentgroup')

,incidentgroup,stopcodedescription,is_malicious_false_alarm
0,False Alarm,False alarm - Good intent,0
2,False Alarm,AFA,0
65,False Alarm,False alarm - Malicious,1
1,Fire,Primary Fire,0
14,Fire,Chimney Fire,0
571,Fire,Secondary Fire,0
3127,Fire,Late Call,0
452853,NULL,NULL,0
11,Special Service,Special Service,0
42,Special Service,Flood call attended - Batch mobilised,0


# Look for predictors

In [30]:
df_features.stopcodedescription.unique()

array(['False alarm - Good intent', 'Primary Fire', 'AFA',
       'Special Service', 'Chimney Fire',
       'Flood call attended - Batch mobilised', 'False alarm - Malicious',
       'Secondary Fire', 'Late Call', 'Use of Special Operations Room',
       'NULL'], dtype=object)

In [36]:
df_features[['incidentgroup','stopcodedescription']].drop_duplicates().sort_values(by='incidentgroup')

,incidentgroup,stopcodedescription
0,False Alarm,False alarm - Good intent
2,False Alarm,AFA
65,False Alarm,False alarm - Malicious
1,Fire,Primary Fire
14,Fire,Chimney Fire
571,Fire,Secondary Fire
3127,Fire,Late Call
452853,NULL,NULL
11,Special Service,Special Service
42,Special Service,Flood call attended - Batch mobilised
